In [35]:
%load_ext autoreload
%autoreload 2

[autoreload of custom_graphgym.head.example failed: Traceback (most recent call last):
  File "/home/stu13/s18/lb9849/miniconda3/envs/pyg/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/stu13/s18/lb9849/miniconda3/envs/pyg/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/stu13/s18/lb9849/miniconda3/envs/pyg/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/stu13/s18/lb9849/BioCompute/pytorch_geometric/graphgym/custom_graphgym/head/example.py", line 27, in <module>
    class ExampleGNNEdgeHead(torch.nn.Module):
  File "/home/stu13/s18/lb9849/BioCompute

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import logging
import os
import custom_graphgym  # noqa, register custom modules
import torch
from torch_geometric import seed_everything
from torch_geometric.graphgym.cmd_args import parse_args
from torch_geometric.graphgym.config import (
    cfg,
    dump_cfg,
    load_cfg,
    set_out_dir,
    set_run_dir,
)
from torch_geometric.graphgym.logger import set_printing
from torch_geometric.graphgym.model_builder import create_model
from torch_geometric.graphgym.train import GraphGymDataModule, train
from torch_geometric.graphgym.utils.agg_runs import agg_runs
from torch_geometric.graphgym.utils.comp_budget import params_count
from torch_geometric.graphgym.utils.device import auto_select_device

In [37]:
from torch_geometric.graphgym import register

In [38]:
register.head_dict.keys()

dict_keys(['node', 'link_pred', 'edge', 'graph', 'custom', 'example2', 'example_new_super_new'])

In [39]:
import argparse

class NotebookArgParser:
    def __init__(self, args_str):
        self.args = self.parse_args(args_str)

    def parse_args(self, args_str):
        parser = argparse.ArgumentParser(description='GraphGym')

        # Add command-line arguments
        parser.add_argument('--cfg',
                            dest='cfg_file',
                            type=str,
                            required=True,
                            help='The configuration file path.')
        parser.add_argument('--repeat',
                            type=int,
                            default=1,
                            help='The number of repeated jobs.')
        parser.add_argument('--mark_done',
                            action='store_true',
                            help='Mark yaml as done after a job has finished.')
        parser.add_argument('opts',
                            default=None,
                            nargs=argparse.REMAINDER,
                            help='See graphgym/config.py for remaining options.')

        # Parse the command-line arguments
        args = parser.parse_args(args_str)

        return args

In [40]:
# Emulate command-line arguments using input cells
command = "python main_pyg.py --cfg ./configs/pyg/example_link_debug.yaml --repeat 1"
args_str = command.split()[2:]
args_str


['--cfg', './configs/pyg/example_link_debug.yaml', '--repeat', '1']

In [41]:
# Create a NotebookArgParser instance
notebook_parser = NotebookArgParser(args_str)

# Access parsed arguments
args = notebook_parser.args
print("Parsed Arguments:")
print(f"Configuration File: {args.cfg_file}")
print(f"Repeat: {args.repeat}")
print(f"Mark Done: {args.mark_done}")
print(f"Remaining Options: {args.opts}")

Parsed Arguments:
Configuration File: ./configs/pyg/example_link_debug.yaml
Repeat: 1
Mark Done: False
Remaining Options: []


In [42]:
# Load config file
load_cfg(cfg, args)
#set_out_dir(cfg.out_dir, args.cfg_file)

# Set Pytorch environment
torch.set_num_threads(cfg.num_threads)
dump_cfg(cfg)
# Repeat for different random seeds

In [43]:
cfg.run_dir

'results/link_predict_debug'

In [44]:
cfg.out_dir

'results/link_predict_debug'

In [45]:
set_run_dir(cfg.out_dir)
set_printing()
# Set configurations for each run
cfg.seed = cfg.seed + 1
seed_everything(cfg.seed)
auto_select_device()
# Set machine learning pipeline
# Load the custom GraphGymDataModule

In [46]:
datamodule = register.train_dict["CustomGraphGymDataModule"]()

DOWNLOADING CUSTOM DATASET LOADER
{'train': Data(x=[2708, 1433], edge_index=[2, 8448], edge_label=[8448], edge_label_index=[2, 8448]), 'test': Data(x=[2708, 1433], edge_index=[2, 9502], edge_label=[1054], edge_label_index=[2, 1054]), 'val': Data(x=[2708, 1433], edge_index=[2, 8448], edge_label=[1054], edge_label_index=[2, 1054])}


/home/stu13/s18/lb9849/BioCompute/pytorch_geometric/torch_geometric/data/in_memory_dataset.py:301: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [47]:
datamodule._train_dataloader

LinkNeighborLoader()

In [48]:
datamodule.dataset.num_edges

10556

In [49]:
cfg.train.batch_size = datamodule.dataset.num_edges

In [50]:
for b in datamodule._train_dataloader:
    print(b)

Data(x=[2705, 1433], edge_index=[2, 8448], edge_label=[8448], edge_label_index=[2, 8448], n_id=[2705], e_id=[8448], num_sampled_nodes=[2], num_sampled_edges=[1], input_id=[8448])


In [51]:
cfg.share.dim_in

1433

In [52]:
cfg.share.dim_out = 1 # TODO fix this bug, that happend in set_dataset_info because dataset._data.y might have node labels (not edge labels)

In [53]:
model=create_model()

INITIALIZING CUSTOM EDGE HEAD


In [54]:
register.loader_dict.keys()

dict_keys(['example'])

In [55]:
register.head_dict['custom']

custom_graphgym.head.example.ExampleGNNEdgeHead

In [56]:
model = create_model()
# Print model infos
logging.info(model)
logging.info(cfg)
cfg.params = params_count(model)
logging.info("Num parameters: %s", cfg.params)

INITIALIZING CUSTOM EDGE HEAD
GraphGymModule(
  (model): GNN(
    (encoder): FeatureEncoder()
    (pre_mp): GeneralMultiLayer(
      (Layer_0): GeneralLayer(
        (layer): Linear(
          (model): Linear(1433, 64, bias=True)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (mp): GNNStackStage(
      (layer0): GeneralLayer(
        (layer): GCNConv(
          (model): GCNConv(64, 64)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
      (layer1): GeneralLayer(
        (layer): GCNConv(
          (model): GCNConv(64, 64)
        )
        (post_layer): Sequential(
          (0): ReLU()
        )
      )
    )
    (post_mp): ExampleGNNEdgeHead(
      (layer_post_mp): MLP(
        (model): Sequential(
          (0): Linear(
            (model): Linear(64, 64, bias=True)
          )
        )
      )
    )
  )
)
accelerator: cpu
benchmark: False
bn:
  eps: 1e-05
  mom: 0.1
cfg_dest: config.yaml
custom_metr

In [57]:
cfg.accelerator = 'cpu'
cfg.devices = 1

In [58]:
# for b in datamodule.train_dataloader():
#     print(b)


In [59]:
# b = next(iter(datamodule.test_dataloader()))

In [60]:
# from torch_geometric.loader import LinkNeighborLoader

# lb = LinkNeighborLoader(data=datamodule.splits['train'],
#             num_neighbors=[-1],
#             batch_size=8448,
#             edge_label_index= datamodule.splits['train'].edge_label_index,
#             edge_label= datamodule.splits['train'].edge_label,
#             shuffle=True,
#             num_workers=cfg.num_workers,
#             pin_memory=True
#         )

In [61]:
# for b in lb:
#     print(b)

In [62]:
# datamodule.splits['test'].edge_label

In [63]:
cfg.share.num_splits = 3 # TODO fixx thisss!

In [64]:
cfg.dataset.task_type

'classification'

In [65]:
cfg.custom_metrics

[]

In [66]:
register.head_dict.keys()

dict_keys(['node', 'link_pred', 'edge', 'graph', 'custom', 'example2', 'example_new_super_new'])

In [33]:
register.train_dict.keys()

dict_keys(['CustomGraphGymDataModule', 'train_pl', 'example'])

In [34]:
# Call the custom training function
register.train_dict["train_pl"](model, datamodule, logger=True)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/stu13/s18/lb9849/miniconda3/envs/pyg/lib/python3.9/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Missing logger folder: results/link_predict_debug/lightning_logs

  | Name  | Type | Params
-------------------------------
0 | model | GNN  | 104 K 
-------------------------------
104 K     Trainable params
0         Non-trainable params
104 K     Total params
0.417     Total estimated model params size (MB)


Sanity Checking DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 98.99it/s]

/home/stu13/s18/lb9849/miniconda3/envs/pyg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


val: {'epoch': 0, 'loss': 0.7329, 'lr': 0.01, 'params': 104256, 'time_iter': 0.0262, 'accuracy': 0.5, 'precision': 0.5, 'recall': 1.0, 'f1': 0.6667, 'auc': 0.6768}
                                                                           

/home/stu13/s18/lb9849/miniconda3/envs/pyg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/home/stu13/s18/lb9849/miniconda3/envs/pyg/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  5.78it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 1/1 [00:00<00:00,  4.16it/s, v_num=0]


/home/stu13/s18/lb9849/miniconda3/envs/pyg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 21.91it/s]
